In [1]:
import torch
import glob
ckpt = torch.load('trained-models/adabound_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0_89-1/tf_efficientnet_b4_ns_bitempered_smooth=0.05_val_loss=0.104_val_acc=0.884_fold2.ckpt')
print(ckpt['callbacks'])

{<class 'pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint'>: {'monitor': 'val_loss', 'best_model_score': tensor(0.1036, device='cuda:0'), 'best_model_path': '/opt/favordata/AI/Felix/kaggle-cassava/trained-models/adabound_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0/tf_efficientnet_b4_ns_bitempered_smooth=0.05_val_loss=0.104_val_acc=0.884_fold2.ckpt', 'current_score': tensor(0.1036, device='cuda:0'), 'dirpath': './runs/adabound_coswarm_bnf_bitemp_smooth_weighted_t1=0.3_t2=1.0/default/version_3/checkpoints'}, <class 'pytorch_lightning.callbacks.early_stopping.EarlyStopping'>: {'wait_count': 0, 'stopped_epoch': 0, 'best_score': tensor(0.1063, device='cuda:0'), 'patience': 6}}


In [6]:
import sys
sys.path.append('../pytorch-image-models')
import timm
timm.create_model('resnext101_32x4d', pretrained=True)

Pretrained model URL is invalid, using random initialization.


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act3): ReLU(inplace=True)
      (do

In [3]:
torch.hub.list('rwightman/pytorch-image-models')

Using cache found in /opt/favordata/.cache/torch/hub/rwightman_pytorch-image-models_master


['adv_inception_v3',
 'cspdarknet53',
 'cspdarknet53_iabn',
 'cspresnet50',
 'cspresnet50d',
 'cspresnet50w',
 'cspresnext50',
 'cspresnext50_iabn',
 'darknet53',
 'densenet121',
 'densenet121d',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenet264',
 'densenet264d_iabn',
 'densenetblur121d',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dpn107',
 'dpn131',
 'dpn68',
 'dpn68b',
 'dpn92',
 'dpn98',
 'eca_vovnet39b',
 'ecaresnet101d',
 'ecaresnet101d_pruned',
 'ecaresnet18',
 'ecaresnet50',
 'ecaresnet50d',
 'ecaresnet50d_pruned',
 'ecaresnetlight',
 'ecaresnext26tn_32x4d',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_pruned',
 'efficientnet_b3a',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_b8',


# WeightedRandomSampler

In [ ]:
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np
s = np.array(list(WeightedRandomSampler([0.1, 0.9, 0.4, 3.0, 0.6], 20000, replacement=True)))
from collections import Counter
counter = Counter(s)
print(counter)
for k,v in counter.items():
    counter[k] = v/20000
for i in range(0, 5):
    print(i, counter[i])

In [ ]:
import pandas as pd
folds = pd.read_csv('./trained-models/sgd_onecycle_bnfrozen_smoothed/folds.csv')
values = folds.label.values

classcounts = Counter(values)
classcounts

In [ ]:
train_df = folds.iloc[folds[folds.fold != 0].index].reset_index(drop=True)
print(len(train_df))
target = train_df.label.values
print('target classes', np.unique(target))

from sklearn.utils.class_weight import compute_class_weight
w = compute_class_weight({0:3,
                          1:2.,
                          2:2.3,
                          4:2.3}, np.unique(target), target)


class_sample_count = np.unique(target, return_counts=True)[1]
print(class_sample_count)
class_sample_count[0] *= 3
class_sample_count[1] *= 2
class_sample_count[2] *= 2.3
class_sample_count[4] *= 2.3

print(class_sample_count)

weight = 1. / class_sample_count
print('w', weight)
samples_weight = weight[target] # unpacks
print(len(samples_weight))
samples_weight = torch.from_numpy(samples_weight)

sampler = WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)

sampled = np.array(list(sampler))
sampled

# smooth cross entropy

In [ ]:
a = np.array([[1,2,3],
             [4,5,6]])
a.sum(axis=0), a.sum(axis=1)

In [ ]:
x = torch.tensor([[0.5, 0.25, 0.75],
                  [0.2, 0.7, 0.1],
                  [0.1, 0.1, 0.8]])
target = torch.tensor([0,2,1])

weight = torch.zeros_like(x)
weight.fill_(0.05 / (3 - 1))
print(weight) # defaults

# randomly scatter self.confidence at indexes across each col dimension
print(weight.scatter_(1, target.unsqueeze(1), 0.95))

# each row contains confidences for the given sample
# each column is  a class

dist_lsm_input = -weight * x
torch.mean(torch.sum(dist_lsm_input, dim=1)) # sum across the columns (each sample's predictions)

In [ ]:
from loss_functions import bi_tempered_logistic_loss 

device = "cpu"

activations = torch.FloatTensor([[-0.5,  0.1,  2.0],
                                [0.1,2,3]]).to(device)
labels = torch.FloatTensor([[0.2, 0.5, 0.3],
                           [0.1,0.2,0.3]]).to(device)

# The standard logistic loss is obtained when t1 = t2 = 1.0
loss = bi_tempered_logistic_loss(activations=activations, labels=labels, t1=1.0, t2=1.0)
print("Loss, t1=1.0, t2=1.0: ", loss)

loss = bi_tempered_logistic_loss(activations=activations, labels=labels, t1=0.7, t2=1.3)
print("Loss, t1=0.7, t2=1.3: ", loss)